In [1]:
import os
import re
import pymatgen as mg
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
import numpy as np
import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import time

In [2]:
torch.set_default_dtype(torch.float64)

torch.set_printoptions(precision=8)

patt_xrd = xrd.XRDCalculator('CuKa')

train_path='/home/hjj/Desktop/GAN-cpx222/train/'

test_path='/home/hjj/Desktop/GAN-cpx222/test/'

global sample_num, rmat_num, series_num
sample_num=1 #output of G
rmat_num=28  #row nums of the matrix for the input of CNN 
series_num=5#input of D



In [3]:
def get_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    energy=round(np.float64(float(energy_string)),5)
    return energy

def linear_transform(energy):
    global extend_num, move_num
    energy_transform=(energy-move_num)*extend_num
    return energy_transform
def inverse_transform(energy_transform):
    global extend_num, move_num
    energy=energy_transform/extend_num+move_num
    return energy
def get_energy_per_atom(energy):
    energy_per_atom=energy/atoms_num
    return energy_per_atom

In [4]:
global extend_num, move_num

In [5]:
extend_num=1000

In [6]:
move_num=get_energy(train_path+'00000/')
print(move_num)

-122.69044


In [7]:
def random_xxpsk(file_path):
    folder=np.random.choice(glob.glob(file_path +"*"))
    #pos_name=folder+'/POSCAR'
    #out_name=folder+'/OUTCAR'
    return folder

def tomgStructure(folder):
    POSfile=folder+'/CONTCAR'      
    R_mgS=mg.Structure.from_file(POSfile)
    return R_mgS

###
##input_data_to_model
###
def get_xrdmat3(mgStructure):
    global rmat_num
    xrd_data4 =patt_xrd.get_pattern(mgStructure)
    #xrd_data4.y=xrd_data4.y-base_pxrd.y
    i_column = rmat_num
    xxx=[]
    yyy=[]
    mat4=[]
    xrd_i=len(xrd_data4)
    for i in range(xrd_i):
        if xrd_data4.y[i] > 0.1 and xrd_data4.y[i] < 70:
            xxx.append(xrd_data4.x[i])
            yyy.append(xrd_data4.y[i])
    mat4.append(np.asarray(xxx))
    mat4.append(np.asarray(yyy))
    mat4=np.asarray(mat4)
    
    xrd_x=[]
    xrd_y=[]
    xrd_mat4=[]
    xrow=len(mat4[0])
    
    if xrow < i_column:
        for i in mat4[0]:
            xrd_x.append(i)
        for j in mat4[1]:
            xrd_y.append(j)
        for i in range(0,i_column-xrow):
            xrd_x.append(0)
            xrd_y.append(0)
        xrd_x=np.asarray(xrd_x)
        xrd_y=np.asarray(xrd_y)
    if xrow > i_column:
        xrd_x=mat4[0][:i_column]
        xrd_y=mat4[1][:i_column]
    if xrow == i_column:
        xrd_x= mat4[0]
        xrd_y= mat4[1]
        
    
    xrd_x=np.sin(np.dot(1/180*np.pi,xrd_x))
    xrd_y=np.sqrt(1/100,xrd_y)
    xrd_mat4.append(xrd_x)
    xrd_mat4.append(xrd_y)
    xrd_mat4=np.array(xrd_mat4)
    return xrd_mat4
###
##input_data_as_knowlegde
###
'''
def get_Gibbs(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    Gibbs=np.float64(float(energy_string))
    Gibbs=round(Gibbs,6)
    return Gibbs
'''
##
###
def get_atoms_num(folder2):
    xxx=tomgStructure(folder2)
    anum=len(xxx.sites)
    return anum


###
##input_data_for_G
###
def GANs_Gmat(Random_Structure):
    global rmat_num
    RS_xrdmat = get_xrdmat3(Random_Structure)
    multimat3_RS =  np.zeros((rmat_num,rmat_num),dtype='float32')
    multimat3_RS = np.asarray((np.dot(RS_xrdmat.T, RS_xrdmat)))
    return multimat3_RS

In [8]:
class GNet(nn.Module):
    
    def __init__(self, input_size=(sample_num,28,28)):
        super(GNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(#(3,28,28)
                in_channels=sample_num,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(32,28,28)
            nn.ReLU(),#->(32,28,28)
            nn.MaxPool2d(kernel_size=2),
        )#->(#->(32,14,14))
        self.conv2=nn.Sequential(#->(32,14,14))
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(64,14,14)
            nn.ReLU(),#->(64,14,14)
            nn.MaxPool2d(kernel_size=2),#->(64,7,7)
        )
        self.out=nn.Sequential(
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128,sample_num),            
        )
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x) #batch(64,7,7)
        x=x.view(x.size(0),-1) #(batch, 64*7*7)
        output=torch.unsqueeze(self.out(x),dim=0)
        return output

class DNet(nn.Module):
    def __init__(self):
        super(DNet, self).__init__()
        self.Dlstm=nn.LSTM(
            input_size=series_num,
            hidden_size=32,
            num_layers=1,
            batch_first=True,
        )
        self.out=nn.Sequential(
            nn.Linear(32,10),
            nn.ReLU(),
            nn.Linear(10,1),
            nn.Sigmoid(),
        )
        #nn.Linear(32,1)
        #nn.Relu
        #nn.Linear
        #nn.Sigmoid
        
    def forward(self,x):
        D_out,(h_n,h_c)=self.Dlstm(x,None)
        out = self.out(D_out[:,-1,:]) #(batch,time step,input)   
        return out

In [9]:
mat_Gl=[]    
mat_Dl=[]
pre_dd=[]
pre_gg=[]
error_test=[]
error_train=[]


In [14]:
G1=GNet()
D1=DNet()

In [15]:
opt_D1=torch.optim.Adam(D1.parameters(),lr=0.1)
opt_G1=torch.optim.Adam(G1.parameters(),lr=0.1)

In [12]:
train_series=[]
for i in range(series_num):
    path_s=random_xxpsk(train_path)
    ee1=get_energy(path_s)
    ee1=linear_transform(ee1)
    train_series.append(ee1)

In [16]:
file_path=train_path
tfset=[]
for step in range(1,4):       
 
    sample_path=[]
    for i in range(1,sample_num + 1):
        path_ = random_xxpsk(file_path)
        sample_path.append(path_)
        tfset.append(path_)
    E_Gibbs=0
    for path1_ in sample_path:
        try:
            total_energy=get_energy(path1_)
            E_Gibbs=linear_transform(total_energy)
            #print(samp_Gibbs)
        except:
            print(path1_)
         
        train_series.pop(-1)
        train_series.append(E_Gibbs)
        
        
    input_series_D=np.asarray(train_series,dtype=np.float64)       
    input_series_D=Variable(torch.from_numpy(input_series_D[np.newaxis,np.newaxis,:]),requires_grad=True)
    
    prob_Tfactor_mat0=D1(input_series_D)
    pre_dd.append(prob_Tfactor_mat0.data.numpy().mean())
    
    g_in=[]
    for path2_ in sample_path:
        path2_=str(path2_)                
        
        try:
            tomgS=tomgStructure(path2_)
            #print(tomgS)
            gin=GANs_Gmat(tomgS)
            
            #print(gin)
        except:
            pass
        g_in.append(gin)
       
    g_in=np.asarray(g_in)
    g_in=g_in[np.newaxis,:,:,:] 
    g_in=np.asarray(g_in,dtype=np.float64) 
    g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
    Gout=G1(g_in)
    Gout=round(Gout.data.numpy().mean(),6)
    train_series.append(Gout)
    train_series.pop(0)
        
    input_series_D=np.asarray(train_series,dtype=np.float64)       
    input_series_D=Variable(torch.from_numpy(input_series_D[np.newaxis,np.newaxis,:]),requires_grad=True)
    
    
    prob_G1_mat1=D1(input_series_D)
    pre_gg.append(prob_G1_mat1.data.numpy().mean())
    
    D1_loss=-torch.mean(torch.log(prob_Tfactor_mat0)+torch.log(1.-prob_G1_mat1))
    dd=D1_loss.data.numpy().mean()
    mat_Dl.append(dd)
    
    G1_loss=torch.mean(torch.log(1.-prob_G1_mat1))
    gg=G1_loss.data.numpy().mean()
    mat_Gl.append(gg)
    
    if step%3==0:
        opt_D1.zero_grad()
        D1_loss.backward(retain_graph=True)
        opt_D1.step()
        
        opt_G1.zero_grad()
        G1_loss.backward()
        opt_G1.step()
    else:
        opt_D1.zero_grad()
        D1_loss.backward()
        opt_D1.step()
    


    if step%1==0:
        print(step)
        print('error: ',abs(inverse_transform(Gout)-E_Gibbs))
        
        print(dd)
        print(gg)
        print(prob_Tfactor_mat0.data.numpy().mean())
        print(prob_G1_mat1.data.numpy().mean())
        
        
        
        
    
    
    if step%1==0:
        print('----------------------------------test-------------------------')
        file_path_test=test_path
        E_Gibbs_1=[]
        E_Gmodel_1=[]
        for step_test in range(8):
            sample_path=[]
            for i in range(1,sample_num + 1):
                path_ = random_xxpsk(file_path_test)
                sample_path.append(path_)
            
            for path1_ in sample_path:
                try:
                    total_energy=get_energy(path1_)
                    #tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
                except:
                    print(path1_)
                    
            #tt_energy=inverse_transform(float(tt_energy))
            print('DFT',total_energy)
            E_Gibbs_1.append(total_energy)
        
                   

        #print(tfactor.shape)
    
            g_in=[]
            for path2_ in sample_path:
                path2_=str(path2_)                
        
                try:
                    tomgS=tomgStructure(path2_)
            #print(tomgS)
                    gin=GANs_Gmat(tomgS)
                    
            #print(gin)
                except:
                    pass
                g_in.append(gin)
       
            g_in=np.asarray(g_in)
            g_in=g_in[np.newaxis,:,:,:] 
            g_in=np.asarray(g_in,dtype=np.float64) 
            g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
            Gout=G1(g_in)
        
        #print(Gout.shape)
    
            G_data=round(inverse_transform(Gout.data.numpy().mean()),6)
            print('G_predict',G_data)
            E_Gmodel_1.append(G_data)

        
            
            
            print('error',abs(total_energy-G_data))
            
        print('------------------end-test----------------------------')
        xxx=abs(abs(np.asarray(E_Gibbs_1))-abs(np.asarray(E_Gmodel_1))).mean()
        print(xxx)
        error_test.append(xxx)
        
        X=np.asarray(E_Gibbs_1)
        Y=np.asarray(E_Gmodel_1)

        xbar=X.mean()
        ybar=Y.mean()
        SSR=0
        varX=0
        varY=0
        for i in range(len(X)):
            diffxxbar=X[i]-xbar
            diffyybar=Y[i]-ybar
            SSR+=(diffxxbar*diffyybar)
            varX+=diffxxbar**2
            varY+=diffyybar**2
    
        SST=math.sqrt(varX+varY)
        R2=(SSR/SST)**2
        print("R2:",R2)
        #r2.append(R2)
    else:
        pass
    


1
error:  122.690469642
1.4036077805291791
-0.7017923350529128
0.49568459808649185
0.5043039463002251
----------------------------------test-------------------------
DFT -122.69044
G_predict -122.69047
error 3.000000000952241e-05
DFT -122.69632
G_predict -122.690468
error 0.005852000000004409
DFT -122.70303
G_predict -122.690468
error 0.012562000000002627
DFT -122.70289
G_predict -122.690468
error 0.012422000000000821
DFT -122.69426
G_predict -122.690469
error 0.003791000000006761
DFT -122.70242
G_predict -122.690469
error 0.011951000000010481
DFT -122.69223
G_predict -122.690468
error 0.0017619999999993752
DFT -122.69223
G_predict -122.690468
error 0.0017619999999993752
------------------end-test----------------------------
0.0062665000000041715
R2: 4.452499975085662e-13


NameError: name 'r2' is not defined

In [17]:

E_Gibbs_test=[]
E_Gmodel_test=[]
abserrset=[]
MSEset=[]
err0set=[]
testfile=[]
for m1,n1,fname in os.walk(test_path):
    for ieach in n1:
        ieach=test_path+ieach
        testfile.append(ieach)
start=time.time()        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        #GGG=inverse_transform(GGG)
        E_Gibbs_test.append(GGG)
        
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_test.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        mse=(G_data-GGG)**2
        abserrset.append(abserr)
        MSEset.append(mse)
        err0=abs(abserr/GGG)
        err0set.append(err0)
    except:
        print(path_)
end=time.time()
print(end-start)

1276.8950204849243


In [18]:
print(np.asarray(abserrset).mean())

print(np.asarray(MSEset).mean())

print(np.sqrt(np.asarray(MSEset).mean()))

0.008667805324455304
9.93495312655758e-05
0.00996742350186726


In [21]:
E_Gibbs_t=[]
E_Gmodel_t=[]
abs_t_errset=[]
err_t_0set=[]
tMSEset=[]
testfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        testfile.append(ieach)

In [22]:
print(testfile)

['/home/hjj/Desktop/GAN-cpx222/train/00003', '/home/hjj/Desktop/GAN-cpx222/train/00000', '/home/hjj/Desktop/GAN-cpx222/train/00002']


In [23]:

#'''        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        
        #GGG=get_energy_per_atom(GGG)
        E_Gibbs_t.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_t.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        tmse=(G_data-GGG)**2
        tMSEset.append(tmse)
        abs_t_errset.append(abserr)
        err0=abs(abserr/GGG)
        err_t_0set.append(err0)
    except:
        print(path_)

In [24]:
print(np.asarray(abs_t_errset).mean())

print(np.asarray(tMSEset).mean())

0.0033637176164470852
3.3546713363676656e-05


In [ ]:
print(np.sqrt(np.asarray(tMSEset).mean()))

In [19]:
torch.save(G1.state_dict(),"/home/hjj/Desktop/GAN-cpx222/G1_cpx222_526-t3-s5.pkl") 
torch.save(D1.state_dict(),"/home/hjj/Desktop/GAN-cpx222/D1_cpx222_526-t3-s5.pkl")

In [ ]:
'''
sample_path=[]
sample_path.append('/home/mii/Desktop/cpx222/train/00000')
g_in=[]
for path2_ in sample_path:
    path2_=str(path2_)                
        
    try:
        tomgS=tomgStructure(path2_)
            #print(tomgS)
        gin=GANs_Gmat(tomgS)
                
            #print(gin)
    except:
        pass
    g_in.append(gin)
       
g_in=np.asarray(g_in)
g_in=g_in[np.newaxis,:,:,:] 
g_in=np.asarray(g_in,dtype=np.float64) 
g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
Gout=G1(g_in)
        
        #print(Gout.shape)
    
G_data=round(inverse_transform(Gout.data.numpy().mean()),6)
print('G_predict',G_data)
'''